In [22]:
%load_ext autoreload
%autoreload 2

import pyBigWig
import pyranges as pr

import sys
sys.path.append('/home/users/pjh/scripts/python_genome_package_dev/')

import handygenome.pyranges_helper as pyranges_helper

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [23]:
bw = pyBigWig.open('/home/users/pjh/practice/bigwig/LU-14.normal.100kbin.processed.bw')

In [144]:
gr = pr.PyRanges(pd.read_csv(
    '/home/users/pjh/practice/bigwig/LU-14.normal.100kbin.processed.bed',
    sep='\t',
    names=['Chromosome', 'Start', 'End', 'depth'],
))

In [145]:
PANEL_REGION_PATH = '/home/users/team_projects/Lung_Cancer_Panel_data/03_Data_from_YTKim/metadata/panel_bait_design/custom/exon_intron_targetregions_merged.bed'
panelregion_gr = pr.read_bed(PANEL_REGION_PATH)
panelregion_gr.Chromosome = 'chr' + panelregion_gr.Chromosome.astype(str)

In [24]:
bw.header()

{'version': 4,
 'nLevels': 5,
 'nBasesCovered': 3095677412,
 'minVal': 0,
 'maxVal': 3279,
 'sumData': 112981193091,
 'sumSquared': 9814727078973}

In [25]:
bw.values('chr1', 0, 4)

[63.36000061035156, 63.36000061035156, 63.36000061035156, 63.36000061035156]

In [28]:
bw.stats(
    'chr1', 
    [1570592] * 2, 
    [1571228] * 2, 
    type='mean',
)

RuntimeError: The start coordinate must be a number!

In [26]:
help(bw.stats)

Help on built-in function stats:

stats(...) method of pyBigWig.bigWigFile instance
    Return summary statistics for a given range. On error, this function throws a
    runtime exception.
    
    Positional arguments:
        chr:   Chromosome name
    
    Keyword arguments:
        start: Starting position
        end:   Ending position
        type:  Summary type (mean, min, max, coverage, std), default 'mean'.
        nBins: Number of bins into which the range should be divided before
               computing summary statistics. The default is 1.
        exact: By default, pyBigWig uses the same method as Kent's tools from UCSC
               for computing statistics. This means that 'zoom levels' may be
               used, rather than actual values (please see the pyBigWig repository
               on github for further information on this). To avoid this behaviour,
               simply specify 'exact=True'. Note that values returned will then
               differ from what U

In [146]:
%%timeit

data1 = [
    bw.stats(row[0], row[1], row[2], type='mean', exact=True)[0]
    for idx, row in panelregion_gr.df.iterrows()
]

193 ms ± 4 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [147]:
%%timeit

joined_gr = pyranges_helper.join(panelregion_gr, gr, how='left', merge='mean') 
data2 = joined_gr.depth

537 ms ± 15.9 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [12]:
import Bio.SeqUtils
type(Bio.SeqUtils.gc_fraction('NNNA'))

float

In [5]:
outfile_path = '/home/users/pjh/practice/bigwig/test_output.bigwig'
bw = pyBigWig.open(outfile_path, 'w')
bw.addHeader([('chr1', 105), ('chr2', 50)])

bw.addEntries(
    'chr1',
    0,
    # values=[float(x) for x in range(11)],
    values=tuple([float(x) for x in range(11)]),
    # values=array.array('f', [float(x) for x in range(11)]),
    span=10,
    step=10,
)
bw.close()

RuntimeError: You must provide a valid set of entries. These can be comprised of any of the following: 
1. A list of each of chromosomes, start positions, end positions and values.
2. A list of each of start positions and values. Also, a chromosome and span must be specified.
3. A list values, in which case a single chromosome, start position, span and step must be specified.


In [3]:
import array
array.array('f', [1, 2, 3, 4])[0]

1.0

In [7]:
help(bw.addEntries)

Help on built-in function addEntries:

addEntries(...) method of pyBigWig.bigWigFile instance
    Adds one or more entries to a bigWig file. This returns nothing, but throws a
    runtime exception on error.
    
    This function always accepts an optional 'validate' option. If set to 'True',
    which is the default, the input entries are checked to ensure that they come
    after previously entered entries. This comes with significant overhead, so if
    this is instead 'False' then this validation is not performed.
    
    There are three manners in which entries can be stored in bigWig files.
    
    
    bedGraph-like entries (12 bytes each):
    
    Positional arguments:
        chrom:  A list of chromosome. These MUST match those added with addHeader().
        starts: A list of start positions. These are 0-based.
    
    Keyword arguments:
        ends:   A list of end positions. These are 0-based half open, so a start of
                0 and end of 10 specifies the first

In [6]:
list(range(0, 101, 10))

[0, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100]

In [129]:
bw = pyBigWig.open(outfile_path)

In [130]:
bw.intervals('chr1')

((0, 10, 0.0),
 (10, 20, 1.0),
 (20, 30, 2.0),
 (30, 40, 3.0),
 (40, 50, 4.0),
 (50, 60, 5.0),
 (60, 70, 6.0),
 (70, 80, 7.0),
 (80, 90, 8.0),
 (90, 100, 9.0),
 (100, 110, 10.0))

In [142]:
bw.values('chr1', 103, 105)

[10.0, 10.0]

In [19]:
bw = pyBigWig.open('/home/users/pjh/tmp/gc100bin.bigwig')
bw.values('5', 1_000_000, 1_000_001, numpy=True)

array([0.51], dtype=float32)

In [18]:
import Bio.SeqUtils

Bio.SeqUtils.gc_fraction(fasta.fetch('5', 1_000_000, 1_000_000 + 100))

0.51

In [5]:
import handygenome.cnv.misc as cnvmisc
import pysam

outfile_path = '/home/users/pjh/tmp/gc100bin.bigwig'
fasta = pysam.FastaFile('/home/users/data/01_reference/human_g1k_v37/human_g1k_v37.fasta')
cnvmisc.make_gc_bigwig(outfile_path, fasta, binsize=100, verbose=True)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
X
Y
MT
GL000207.1
GL000226.1
GL000229.1
GL000231.1
GL000210.1
GL000239.1
GL000235.1
GL000201.1
GL000247.1
GL000245.1
GL000197.1
GL000203.1
GL000246.1
GL000249.1
GL000196.1
GL000248.1
GL000244.1
GL000238.1
GL000202.1
GL000234.1
GL000232.1
GL000206.1
GL000240.1
GL000236.1
GL000241.1
GL000243.1
GL000242.1
GL000230.1
GL000237.1
GL000233.1
GL000204.1
GL000198.1
GL000208.1
GL000191.1
GL000227.1
GL000228.1
GL000214.1
GL000221.1
GL000209.1
GL000218.1
GL000220.1
GL000213.1
GL000211.1
GL000199.1
GL000217.1
GL000216.1
GL000215.1
GL000205.1
GL000219.1
GL000224.1
GL000223.1
GL000195.1
GL000212.1
GL000222.1
GL000200.1
GL000193.1
GL000194.1
GL000225.1
GL000192.1
